## Intro to Conversational AI and Rasa

In [ ]:
!pip install nest_asyncio
!pip install rasa

### Start the event loop

Running asynchronous Rasa code in Jupyter Notebooks requires an extra requirement, since Jupyter Notebooks already run on event loops. Run the cell below to start the event loop:

In [ ]:
import nest_asyncio

nest_asyncio.apply()
print("Event loop ready.")

### Create a new project and train an example assistant

Create a new project called **rasabot**. The function below will create a new project with example training data and most important files. It replicates the behaviour of Rasa CLI function **rasa init**.


In [ ]:
import rasa
from rasa.cli.scaffold import create_initial_project
import os

project = "rasabot"
create_initial_project(project)

# move into project directory and show files
os.chdir(project)
print(os.listdir("."))

# define the project files
config = "config.yml"
training_files = "data/"
domain = "domain.yml"
output = "models/"

#train the example assistant
model_path = rasa.train(domain, config, [training_files], output)

### Have a chat with an assistant

The function above creates an assistant called moodbot which asks how you feel and if you are sad it tries to cheer your up with a picture of a cute tiger cub. You can test an assistant using the functions below. To stop the chat type '/stop' in a chat window.
The code cell below replicates the behaviour of the Rasa CLI function **rasa shell**.

In [ ]:
from rasa.jupyter import chat
chat(model_path)

## Creating the NLU training data

Now, we learn about the format of the NLU training data and create the necessary intent for our assistant. By running the fell below, you will update the training dataset with the new created intents. You are encouraged to add more training examples to existing intents or create completely new ones.

In [ ]:
nlu_md = """
## intent:greet
- hey
- hello
- hi
- good morning
- good evening
- hey there

## intent:goodbye
- bye
- goodbye
- see you around
- see you later

## intent:inform
- [Sitka](location)
- [Juneau](location)
- [Virginia](location)
- [Cusseta](location)
- [Chicago](location)
- [Tuscon](location)
- [Columbus](location)
- [San Francisco](location)

## intent:search_provider
- I need a [hospital](facility_type)
- find me a nearby [hospital](facility_type)
- show me [home health agencies](facility_type)
- [hospital](facility_type)
- find me a nearby [hospital](facility_type) in [San Francisco](location)
- I need a [home health agency](facility_type)

## intent:affirm
- yes
- indeed
- of course
- that sounds good
- correct

## intent:deny
- no
- never
- I don't think so
- don't like that
- no way
- not really

## intent:mood_great
- perfect
- very good
- great
- amazing
- wonderful
- I am feeling very good
- I am great
- I'm good

## intent:mood_unhappy
- sad
- very sad
- unhappy
- bad
- very bad
- awful
- terrible
- not very good
- extremely sad
- so sad
"""

%store nlu_md > data/nlu.md